In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Задание 1

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data', header=None)
df.iloc[:5,:]

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [ ]:
cat_columns = df.iloc[:, 1:23]
for column in cat_columns:
  distribution = df.groupby([0, column]).size()
  print(distribution)
  plt.figure(figsize = (10,10))
  sns.countplot(data=df, x=column, hue=df[0])
  plt.show()
  plt.close()

NameError: ignored

# Задание 2

In [ ]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.naive_bayes import MultinomialNB

inform_variable = None
best_accuracy = 0
y = df.iloc[:, 0]

for i in range(1,23):

  X = df.iloc[:, i]

  X = X.to_frame().values.reshape(-1,1)

  encoder = OneHotEncoder()
  encoded_X = encoder.fit_transform(X)
  #print(encoded_X)

  classifier = CategoricalNB()
  classifier.fit(encoded_X.toarray(), y)
  print(encoded_X.toarray().shape)

  class_probabilities = classifier.predict_proba(encoded_X.toarray())
  #print(class_probabilities.shape)

  predicted_classes = classifier.classes_[class_probabilities.argmax(axis=1)]
  accuracy = accuracy_score(y, predicted_classes)
  print(i, accuracy)

  if accuracy > best_accuracy:
    best_accuracy = accuracy
    inform_variable = i

print(best_accuracy, inform_variable)

(8124, 6)
1 0.5593303791235844
(8124, 4)
2 0.5805022156573116
(8124, 10)
3 0.5952732644017725
(8124, 2)
4 0.7439684884293452
(8124, 9)
5 0.9852289512555391
(8124, 2)
6 0.5034465780403742
(8124, 2)
7 0.6159527326440177
(8124, 2)
8 0.7562776957163959
(8124, 12)
9 0.8050221565731167
(8124, 2)
10 0.5529295913343181
(8124, 5)
11 0.6459871984244214
(8124, 4)
12 0.7735105859182668
(8124, 4)
13 0.7661250615460364
(8124, 9)
14 0.7163958641063516
(8124, 9)
15 0.7144263909404235
(8124, 1)
16 0.517971442639094
(8124, 4)
17 0.5046774987690793
(8124, 3)
18 0.5381585425898572
(8124, 5)
19 0.7754800590841949
(8124, 9)
20 0.8680452978828164
(8124, 6)
21 0.7218119153126539
(8124, 7)
22 0.690300344657804
0.9852289512555391 5


# Задание 3

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score

X = df.iloc[:, 1:23]
y = df.iloc[:, 0]

encoder = OneHotEncoder()
encoded_X = encoder.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(encoded_X, y, test_size=0.2, random_state=42)
classifier = CategoricalNB()
classifier.fit(X_train.toarray(), y_train)

y_pred = classifier.predict(X_test.toarray())

accuracy = accuracy_score(y_test, y_pred)
print(f'Точность классификатора: {accuracy * 100:.2f}%')




Точность классификатора: 93.60%


# Задание 4

In [ ]:
freq_features = []

X = df.iloc[:, 1:23]
y = df.iloc[:, 0]

#разбиваем данные для обучения и для теста
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

data = pd.concat([y_train, X_train], axis = 1)

#находим количество съедобных и несъедобных грибов
sum_e = data[0].value_counts()[0]
sum_p = data[0].value_counts()[1]
sum = sum_e + sum_p

#здесь проиходит обучение
for i in range(1,23):
  f = data.groupby(i)[0].value_counts().unstack().fillna(0) #вычисляем количество грибов в каждом значении каждого признака
  print(f)
  f['e'] = (f['e'] + 1) / (sum_e + 2)
  f['p'] = (f['p'] + 1) / (sum_p + 2)   #вычисляем вероятности каждого значения в каждом классе
  print(f)
  freq_features.append(f)

y_pred = []

X_test.reset_index(drop=True, inplace=True)

#здесь происходит предсказание
for i in range(len(X_test)):
  mushroom = X_test.iloc[i, :] #берем гриб(строку признаков)
  freq_e = 1
  freq_p = 1   # вероятности возникновения классов e и p
  for j in range(1, len(mushroom) + 1):    # проходимся по признакам в строке
    mush_feat = mushroom[j]
    feat = freq_features[j - 1]
    freq_e *= feat.loc[mush_feat, 'e']
    freq_p *= feat.loc[mush_feat, 'p']   # умножаем на вероятность возникновения данного значения признака для каждого класса
  freq_e *= (sum_e / sum)
  freq_p *= (sum_p / sum)    # умножаем на вероятность возникновения класса
  if freq_e > freq_p:
    y_pred.append('e')
  else:
    y_pred.append('p')

accuracy = accuracy_score(y_test, y_pred)
print(f'Точность классификатора: {accuracy * 100:.2f}%')

0       e       p
1                
b   313.0    40.0
c     0.0     3.0
f  1248.0  1224.0
k   192.0   496.0
s    27.0     0.0
x  1585.0  1371.0
0         e         p
1                    
b  0.093258  0.013074
c  0.000297  0.001276
f  0.370953  0.390625
k  0.057321  0.158482
s  0.008316  0.000319
x  0.471042  0.437500
0       e       p
2                
f  1244.0   602.0
g     0.0     3.0
s   913.0  1135.0
y  1208.0  1394.0
0         e         p
2                    
f  0.369765  0.192283
g  0.000297  0.001276
s  0.271458  0.362245
y  0.359073  0.444834
0      e      p
3              
b   44.0   99.0
c   27.0   11.0
e  499.0  699.0
g  820.0  631.0
n  997.0  826.0
p   49.0   69.0
r   12.0    0.0
u   14.0    0.0
w  588.0  258.0
y  315.0  541.0
0         e         p
3                    
b  0.013365  0.031888
c  0.008316  0.003827
e  0.148500  0.223214
g  0.243837  0.201531
n  0.296406  0.263712
p  0.014850  0.022321
r  0.003861  0.000319
u  0.004455  0.000319
w  0.174933  0.082589
y  0.0

# Задание 5

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

for i in range(1,23):
  f = df.groupby(i)[0].value_counts().unstack().fillna(0)
  f['prob'] = (f['p'] + 0.001)/(f['p']+f['e']+0.002)
  P = df[i].map(lambda x: f.at[x,'prob'])
  df['inv_sig%i'%i] = np.log(P) - np.log(1.0 - P)

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 23:], df.iloc[:, 0])

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность классификатора: {accuracy * 100:.2f}%')
print(accuracy)

Точность классификатора: 100.00%
1.0
